In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from sklearn.datasets import load_iris
import math

In [4]:
iris = load_iris()

In [5]:
df = pd.DataFrame(data=np.c_[iris['data'],iris['target']] , columns=iris['feature_names']+['target'])

In [6]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [14]:
class EnTree:
    def __init__(self,depth=0,max_depth=5,target=None):
        self.depth = depth
        self.max_depth = max_depth
        self.childs = []
        self.dataframe = None
        self.target = target
        self.Ret = None
        self.columns = None
    
    def entropy(self,column):
        _ , counts = np.unique(column,return_counts=True)
        s = 0
        for count in counts:
            s+= float(count)/sum(counts) * math.log2(float(count)/sum(counts))
        return -s

    def getIG(self,data,column):
        unique , counts = np.unique(data[column] , return_counts=True)
        ig = 0

        for u,c in zip(unique,counts):
            ig += c / sum(counts) * self.entropy(data[self.target].values[data[column]==u])
        hs = self.entropy(data[self.target])
        return hs-ig

    def split(self,data,column):
        unique , counts = np.unique(data[column] , return_counts=True)
        if(len(counts)>1):
            childs = []
            for u in unique:
                df = pd.DataFrame([],columns=data.columns)
                for each in data.columns:
                    df[each] = data[each].values[data[column]==u]
                childs.append(df)
            return childs
        
        return None
    
    def fit(self,X):
        columns = X.drop(columns=[self.target]).columns
        self.columns = columns
        igs = []
        for each in columns:
            igs.append(self.getIG(X,each))
        
        differ = columns[np.argmax(igs)]
        nodes = self.split(X,differ)

        if nodes is None:
            if(X[self.target].mean() > 0.5):
                self.Ret = 1
            else:
                self.Ret = 0
            
            return
            
        if self.depth == self.max_depth:
            if(X[self.target].mean() > 0.5):
                self.Ret = 1
            else:
                self.Ret = 0

            return 

        for node in nodes:
            child = EnTree(depth=self.depth+1,target=self.target)
            child.dataframe = node
            self.childs.append(child)

        for child in self.childs:
            child.fit(child.dataframe)
        
    
    def predict(self,vals,max_feat_match):
        found = 0
        for level in range(max_feat_match):
            if found == 0:
                for child in self.childs:
                    if found == 0:
                        for row in child.dataframe.iloc:
                            if(sum(row.drop(self.target)==vals) == (len(self.columns)-level)):
                                found = child
                    else:
                        break
            else:
                break
        
        if found==0:
            return self.Ret

        if(found):
            return found.predict(vals,max_feat_match)

            



    



In [15]:
tree = EnTree(target='target')

In [16]:
tree.fit(df)

In [17]:
tree.predict(df.iloc[32].drop('target'),4)

0

In [18]:
df.iloc[32]

sepal length (cm)    5.2
sepal width (cm)     4.1
petal length (cm)    1.5
petal width (cm)     0.1
target               0.0
Name: 32, dtype: float64